<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_23_CrewAI_Legal_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to use Graphlit with CrewAI. Based on this [great blog post](https://farzzy.hashnode.dev/building-a-legal-ai-agent-using-azure-ai-search-azure-openai-llamaindex-and-crewai) and [notebook](https://github.com/farzad528/azure-ai-search-python-playground/blob/main/azure-ai-search-crewai.ipynb) written by Farzad Sunavala, we emulate Farzad's workflow and show how to use Graphlit instead of LlamaIndex, while already leveraging Azure AI Search under the hood.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

For CrewAI, you will need your OpenAI API key.

Assign this property as Colab secret: OPENAI_API_KEY.

For HuggingFace, you will need a HuggingFace token. To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens)

Assign this property as Colab secret: HF_TOKEN.

---

In [1]:
!pip install --upgrade crewai

In [2]:
!pip install --upgrade graphlit-tools[crewai]

In [3]:
!pip install --upgrade datasets

Initialize Graphlit

In [4]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Initialize sample data

In [5]:
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/umarbutler/open-australian-legal-corpus

# Load only the first 100 rows from the JSONL file
dataset_df = pd.read_json("hf://datasets/umarbutler/open-australian-legal-corpus/corpus.jsonl", lines=True, nrows=100)

In [6]:
dataset_df.head()

,version_id,type,jurisdiction,source,mime,date,citation,url,when_scraped,text
0,south_australian_legislation:46593b41d5646f59/...,secondary_legislation,south_australia,south_australian_legislation,application/rtf,2018-06-14,Fences Regulations 2018 (SA),https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:15.691552+10:00,South Australia\nFences Regulations 2018\nunde...
1,south_australian_legislation:2021-01-01/1999.4...,primary_legislation,south_australia,south_australian_legislation,application/rtf,2021-01-01,Federal Courts (State Jurisdiction) Act 1999 (SA),https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:15.730486+10:00,South Australia\nFederal Courts (State Jurisdi...
2,south_australian_legislation:2018-11-01/2005.7...,primary_legislation,south_australia,south_australian_legislation,application/rtf,2018-11-01,Terrorism (Police Powers) Act 2005 (SA),https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:15.817516+10:00,South Australia\nTerrorism (Police Powers) Act...
3,south_australian_legislation:2023-11-23/2021.5...,primary_legislation,south_australia,south_australian_legislation,application/rtf,2023-11-23,Social Workers Registration Act 2021 (SA),https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:15.988610+10:00,South Australia\nSocial Workers Registration A...
4,south_australian_legislation:2023-03-30/2021.1...,secondary_legislation,south_australia,south_australian_legislation,application/rtf,2023-03-30,Primary Industry Funding Schemes (Clare Valley...,https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:16.550214+10:00,South Australia\nPrimary Industry Funding Sche...


In [7]:
# Drop the 'mime' and 'when_scraped' columns
dataset_df = dataset_df.drop(columns=['mime', 'when_scraped'])

# Verify the columns have been dropped
print(dataset_df.columns)

Index(['version_id', 'type', 'jurisdiction', 'source', 'date', 'citation',
       'url', 'text'],
      dtype='object')


In [8]:
# Convert 'date' column to the correct DateTimeOffset string format (ISO 8601 format)
# Assuming that the original date field is in UTC, but if it's not, you might need to adjust the timezone
dataset_df['date'] = dataset_df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

# Ensure other fields are properly cast as strings
dataset_df['type'] = dataset_df['type'].astype(str)
dataset_df['source'] = dataset_df['source'].astype(str)
dataset_df['jurisdiction'] = dataset_df['jurisdiction'].astype(str)
dataset_df['citation'] = dataset_df['citation'].astype(str)
dataset_df['version_id'] = dataset_df['version_id'].astype(str)
dataset_df['url'] = dataset_df['url'].astype(str)
dataset_df['text'] = dataset_df['text'].astype(str)

Load list of documents

In [9]:
import json
from IPython.display import display, Markdown, HTML

# Convert the DataFrame to a JSON string representation
documents_json = dataset_df.to_json(orient="records")

# Load the JSON string into a Python list of dictionaries
documents_list = json.loads(documents_json)

document = documents_list[0]

display(Markdown(f'### Document: [{document["date"]}] {document["version_id"]}'))
display(Markdown(f'**URI** [{document["url"]}] '))
display(Markdown(f'**Type** [{document["type"]}]'))
display(Markdown(f'**Source** [{document["source"]}]'))
display(Markdown(f'**Jurisdiction** [{document["jurisdiction"]}]'))
display(Markdown(f'**Citation** [{document["citation"]}]'))
display(Markdown(document["text"]))

### Document: [2018-06-14T00:00:00.000000Z] south_australian_legislation:46593b41d5646f59/2018.74.un

**URI** [https://www.legislation.sa.gov.au/__legislation/lz/c/r/fences%20regulations%202018/current/2018.74.un.rtf] 

**Type** [secondary_legislation]

**Source** [south_australian_legislation]

**Jurisdiction** [south_australia]

**Citation** [Fences Regulations 2018 (SA)]

South Australia
Fences Regulations 2018
under the Fences Act 1975



Contents
1	Short title
2	Commencement
3	Exempt classes of Land
Schedule 1—Revocation of Fences Regulations 2003
Legislative history



1—Short title
These regulations may be cited as the Fences Regulations 2018.
2—Commencement
These regulations come into operation on the day on which they are made.
3—Exempt classes of Land
Pursuant to section 20(3) of the Fences Act 1975, the following classes of land are exempt from the provisions of that Act:
	(a)	land held by the Commissioner of Highways or any council for the purposes of controlling access to a road or proposed road from land abutting the road or proposed road;
	(b)	land of, or used by, the Crown, an instrumentality or agency of the Crown or a council that is used solely or principally for the purpose of drainage.



Schedule 1—Revocation of Fences Regulations 2003
The Fences Regulations 2003 are revoked.



Legislative history
Notes
	•	For further information relating to the Act and subordinate legislation made under the Act see the Index of South Australian Statutes or www.legislation.sa.gov.au.
Principal regulations
Year|No|Reference|Commencement|
2018|74|Gazette 14.6.2018 p2162 |14.6.2018: r 2|

Define Graphlit helper functions

In [10]:
import base64
import mimetypes
import asyncio
from typing import List, Optional
from tqdm.asyncio import tqdm

async def process_text(document, progress_bar):
    content_id = await ingest_text(document)

    if content_id is not None:
        print(f'Ingested content [{content_id}].')

    progress_bar.update(1)

async def ingest_text(document):
    if graphlit.client is None:
        return;

    try:
        name = document['version_id']
        text = document['text']
        url = document['url']

        # TODO: consider how to assign custom metadata with ingestion, i.e. type, source, jurisdiction, citation

        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_text(name=name, text=text, text_type=enums.TextTypes.PLAIN, uri=url, is_synchronous=True)

        return response.ingest_text.id if response.ingest_text is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model,
        ),
        strategy=input_types.ConversationStrategyInput(
            embedCitations=True
        ),
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.SECTION
        ),
        rerankingStrategy=input_types.RerankingStrategyInput(
            serviceType=enums.RerankingModelServiceTypes.COHERE
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return None, None

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        message = response.prompt_conversation.message.message if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
        citations = response.prompt_conversation.message.citations if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None

        return message, citations
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None, None

# NOTE: these functions are just used to clean-up old data before executing the example
async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

Ingest sample documents

In [11]:
import nest_asyncio

nest_asyncio.apply()

In [12]:
# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

progress_bar = tqdm(total=len(documents_list))

tasks = [process_text(document, progress_bar) for document in documents_list]

await asyncio.gather(*tasks)

print('Ingested all contents.')

Deleted all contents.


  1%|          | 1/100 [00:08<14:50,  8.99s/it]

Ingested content [0685b3ff-7917-4e90-8039-6d91db6de5b1].


  4%|▍         | 4/100 [00:14<04:13,  2.64s/it]

Ingested content [dfaaa3db-59e6-4d37-8f51-91b905f41f5b].
Ingested content [51d0ea27-2ba9-40d0-a57e-6bd0241c6f3e].
Ingested content [2601670e-f5f3-4231-a390-be8e98294a0a].
Ingested content [56992d0a-be16-4b7d-a588-4e99585ec744].


...

Ingested content [0848ad90-7009-40c7-bc01-b55bae260c1b].
Ingested content [1c1bb349-3256-4b90-9e27-e11736a15575].
Ingested content [b77898ff-cd26-4499-8bbb-fe7c52e2aed2].
Ingested content [83941896-114b-43e4-b709-950c712ec91b].
Ingested content [8d431bc0-5072-4bdf-b85c-bfb825336b1c].
Ingested content [b2decebd-4852-4d79-9046-29d2f3832e5f].
Ingested content [d07a7f72-5fad-4b5b-9d32-b4fd19b7d1b1].
Ingested content [a066a137-7b82-4038-8c00-8bff5c9e8d8b].
Ingested content [eee014a2-3776-40e6-860b-c8b1674127f7].
Ingested content [86d38059-7182-4bc5-b29c-9e8f429a81db].
Ingested content [fd38121a-651f-4654-802a-f119cefb8e3f].
Ingested content [b87bcecd-ecc4-4d0f-8803-7ac1abe674a5].
Ingested content [7f93df1a-45e6-4cc6-b553-d1a7960731fc].


 97%|█████████▋| 97/100 [00:44<00:01,  2.63it/s]

Ingested content [1b6caee0-5430-4fa2-b9e3-a9259e63c71c].
Ingested content [9b8d3e95-7ec9-4161-8386-6be041906ebf].
Ingested content [48335112-db0e-4251-a327-cdb9fcd61b77].
Ingested content [a362216e-1251-44a0-b3b1-fb8d5c7bd8e7].
Ingested content [bbf0f056-876f-4d70-8de5-87170f2a824b].
Ingested content [161efdfd-423f-465e-b71f-45af4bd51934].


 99%|█████████▉| 99/100 [00:49<00:00,  1.36it/s]

Ingested content [946b83d3-046f-4e60-9d45-f03500ef7e5c].
Ingested content [8793d7c4-bf03-463f-a0c7-6cd9814cd2f7].
Ingested all contents.


Execute example RAG prompt with citations

In [13]:
import nest_asyncio

nest_asyncio.apply()

In [14]:
import time

# Remove any existing conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()

print('Deleted all conversations and specifications.')

prompt = 'If a Clare Valley winemaker inadvertently underpays their contributions to the Fund, what are the potential consequences, and is there a grace period for making up the shortfall?'

model = enums.OpenAIModels.O1_MINI_128K
#model = enums.OpenAIModels.GPT4O_MINI_128K

specification_id = await create_openai_specification(model)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    conversation_id = await create_conversation(specification_id=specification_id)

    if conversation_id is not None:
        message, citations = await prompt_conversation(conversation_id, prompt)

        if message is not None:
            display(Markdown(f'**Final Response:**\n{message}'))
            print()

        if citations is not None:
            print("\nReference Information:")

            for citation in citations:
                if citation is not None and citation.content is not None:
                    display(Markdown(f'**Citation [{citation.index}]:** {citation.content.name}'))
                    display(Markdown(citation.text))
                    print()


Deleted all conversations and specifications.
Created specification [6bbf8c42-08c5-4cbe-b2e6-86604ab07a65].


100%|██████████| 100/100 [00:59<00:00,  1.36it/s]

**Final Response:**
If a Clare Valley winemaker underpays their contributions to the Fund, they may be deemed in default according to the Primary Industry Funding Schemes (Clare Valley Wine Industry Fund) Regulations 2021. Being in default can lead to exclusion from receiving direct benefits or services funded by the Fund. [1][2]

While the regulations emphasize the importance of timely contributions, there is no explicitly stated grace period for making up the shortfall. It is crucial for winemakers to address any underpayments promptly to restore their standing and avoid exclusion from Fund benefits. [1][2]



Reference Information:


**Citation [1]:** south_australian_legislation:2023-03-30/2021.127.un

South Australia
Primary Industry Funding Schemes (Clare Valley Wine Industry Fund) Regulations 2021
under the Primary Industry Funding Schemes Act 1998
Contents
1	Short title
3	Interpretation
4	Clare Valley Wine Industry Fund
5	Contributions to Fund
6	Refunds of contributions
7	Application of Fund
8	Exchange of information
9	Exclusion from benefits of person in default in relation to contributions
10	False or misleading statements
Schedule 1—Map showing Clare Valley
Schedule 2—Transitional provisions
Part 2—Transitional provisions
2	Transitional provisions
Legislative history
1—Short title
These regulations may be cited as the Primary Industry Funding Schemes (Clare Valley Wine Industry Fund) Regulations 2021.
3—Interpretation
	(1)	In these regulations, unless the contrary intention appears—
Act means the Primary Industry Funding Schemes Act 1998;
Clare Valley means the geographical area in relation to which the geographical indication "Clare Valley" is in force under the Wine Australia Act 2013 of the Commonwealth at the commencement of these regulations (see Schedule 1);
Clare Valley grapes means any variety of grapes grown in the Clare Valley and used or intended to be used for wine;
Clare Valley winemaker means a person who carries on a business of making wine and who processes Clare Valley grapes for that purpose;
default in relation to contributions to the Fund—see subregulation (2);
delivered—see subregulation (3);
Fund—see regulation 4;
prescribed period means each of the following periods:
	(a)	a prescribed period within the meaning of the revoked regulations (up to and including the period of 12 months commencing on 1 June 2021);
	(b)	the period of 12 months commencing on 1 June 2022 or on 1 June in any subsequent year;
revoked regulations means the Primary Industry Funding Schemes (Clare Valley Wine Industry Fund) Regulations 2008.
	(2)	A person is in default in relation to contributions to the Fund if, within the immediately preceding 2 prescribed periods—
	(a)	all or some of the contributions payable to the Fund by the person under these or the revoked regulations have not been paid; or
	(b)	the person has been refunded contributions from the Fund under these or the revoked regulations.
	(3)	For the purposes of these regulations—
	(a)	grapes will be taken to be delivered to a Clare Valley winemaker by a grower of Clare Valley grapes when the winemaker takes possession of the grapes; and
	(b)	if a Clare Valley winemaker processes Clare Valley grapes grown by the winemaker, grapes will be taken to be delivered to the winemaker when the winemaker places the grapes in a container for the purposes of commencing processing of the grapes (including placing the grapes in a container for fermentation or in preparation for crushing or pressing).
4—Clare Valley Wine Industry Fund
	(1)	The Clare Valley Wine Industry Fund (the Fund) established under the revoked regulations continues in existence.
	(2)	The Fund is administered by the Minister.
	(3)	The Fund consists of—
	(a)	the amount held in the Fund at the commencement of these regulations; and
	(b)	contributions paid or collected in accordance with the revoked regulations; and
	(c)	contributions paid or collected in accordance with these regulations; and
	(d)	income of the Fund from investment; and
	(e)	any other sums received by the Minister for payment into the Fund.
5—Contributions to Fund
	(1)	Subject to these regulations, the following contributions are payable to the Minister for payment into the Fund for Clare Valley grapes delivered to a Clare Valley winemaker during a prescribed period commencing on or after 1 June 2022:
	(a)	in the case of a Clare Valley winemaker who has a vineyard, winemaking facility or cellar door in the Clare Valley, the winemaker must contribute whichever of the following is the greater amount:
	(i)	the amount fixed by the Minister, by notice in the Gazette, as the minimum contribution for the period;
	(ii)	an amount being the sum of—
	(A)	the amount fixed by the Minister for the period, by notice in the Gazette, for each tonne of grapes grown by the winemaker delivered during the period; and
	(B)	the amount fixed by the Minister for the period, by notice in the Gazette, for each tonne of grapes grown by a person other than the winemaker delivered during the prescribed period;
	(b)	in the case of any other Clare Valley winemaker, the winemaker must contribute the amount fixed for the period by the Minister, by notice in the Gazette, for each tonne of grapes grown by a person other than the winemaker delivered during the prescribed period;
	(c)	if the grapes are grown by a person other than the winemaker, the grower of the grapes must contribute the amount fixed by the Minister for the period by notice in the Gazette for each tonne of grapes delivered during the prescribed period.
	(2)	The Minister may, by notice in the Gazette, fix maximum amounts that are to be payable for a prescribed period by the grower of grapes or a winemaker under this regulation.
	(3)	The contributions in respect of a prescribed period are payable on or before the last day of the month that immediately follows the prescribed period.
	(4)	Contributions payable by a grower of Clare Valley grapes must be paid on behalf of the grower by the Clare Valley winemaker who purchases the grapes out of the amount payable by the winemaker to the grower for the grapes.

**Citation [2]:** south_australian_legislation:2023-03-30/2021.127.un

(5)	The Minister may, by notice in the Gazette published before the date for payment of contributions for a prescribed period, vary an amount fixed under subregulation (1) or (2) in respect of the prescribed period.
	(6)	If the Minister varies an amount payable under subregulation (1) in respect of a prescribed period after the commencement of the prescribed period, the amount as varied applies only in respect of the part of the prescribed period following the date on which the variation takes effect.
	(7)	A Clare Valley winemaker must—
	(a)	keep proper records relating to the tonnage of Clare Valley grapes delivered to the winemaker, the growers of those grapes and the contributions required to be made (on the winemaker's own behalf and on behalf of growers) in respect of those grapes; and
	(b)	make those records available for inspection at any reasonable time by a person authorised by the Minister for the purpose.
	(8)	A Clare Valley winemaker must, on or before the last day of the month that immediately follows a prescribed period—
	(a)	furnish the Minister with a financial statement relating to the contributions (on the winemaker's own behalf and on behalf of growers) for Clare Valley grapes delivered during that prescribed period that—
	(i)	is in the form, and contains the information, required by the Minister; and
	(ii)	is, if the person has the necessary equipment, in an electronic form acceptable to the Minister; and
	(b)	forward to the Minister, with the financial statement required by paragraph (a), the required contributions (on the winemaker's own behalf and on behalf of growers) for Clare Valley grapes delivered during the prescribed period to which the financial statement relates.
6—Refunds of contributions
	(1)	Refunds of contributions paid under these regulations in respect of Clare Valley grapes delivered during a prescribed period may be claimed by notice in writing to the Minister within the 6 months following that prescribed period as follows:
	(a)	a grower of Clare Valley grapes may claim a refund in respect of contributions paid by the grower or by a Clare Valley winemaker on behalf of the grower;
	(b)	a Clare Valley winemaker may claim a refund in respect of contributions paid on the winemaker's own behalf.
	(2)	A person claiming a refund under subregulation (1) must supply the Minister with evidence acceptable to the Minister of the contributions made by or on behalf of the claimant in respect of which the claim for refund is made.
	(3)	If the person satisfies the Minister that the person is entitled to a refund, the Minister must refund to the person the amount of the contributions paid by or on behalf of the person, together with interest (at the official cash rate) on that amount for the period commencing on the date of payment of the contributions to the Minister and ending on the date of lodgment of the claim for the refund.
	(4)	In this regulation—
official cash rate means the cash rate fixed by the Reserve Bank of Australia and prevailing on the date of lodgment of the claim for the refund.
7—Application of Fund
The Fund may be applied by the Minister for any of the following purposes:
	(a)	payments to a body that, in the opinion of the Minister, represents Clare Valley winemakers or growers of Clare Valley grapes (or both) for 1 or more of the following purposes:
	(i)	the reasonable operating and management expenses of the body;
	(ii)	promoting the Clare Valley wine industry;
	(iii)	undertaking or facilitating research and development, or the collection and dissemination to Clare Valley winemakers and growers of Clare Valley grapes of information, relevant to the Clare Valley wine industry and, in particular, to the improvement of practices in the industry;
	(iv)	the participation of the body in regional, State and national wine industry events;
	(v)	programs designed to encourage communication and cooperation between Clare Valley winemakers and growers of Clare Valley grapes;
	(vi)	other purposes of the body;
	(b)	payments for other purposes for the benefit of the Clare Valley wine industry;
	(c)	payment of the expenses of administering the Fund;
	(d)	repayment of contributions to the Fund under regulation 6.
8—Exchange of information
The Minister may provide a body to which payments are made out of the Fund under regulation 7(a) with information identifying growers of Clare Valley grapes and Clare Valley winemakers who have paid or been refunded contributions under these regulations.
9—Exclusion from benefits of person in default in relation to contributions
A person who is in default in relation to contributions to the Fund is not entitled to receive direct benefits or services funded by payments from the Fund.
10—False or misleading statements
A person must not make a statement that is false or misleading in a material particular (whether by reason of the inclusion or omission of any particular) in any information provided, or record kept, for the purposes of these regulations.
Maximum penalty: $5 000.
Schedule 1—Map showing Clare Valley
The following map is provided for information purposes only.
Schedule 2—Transitional provisions
Part 2—Transitional provisions
2—Transitional provisions
	(1)	Subject to this clause, the revoked regulations, as in force immediately before their revocation under clause 1 of this Schedule, continue to operate in respect of the prescribed period under the revoked regulations that commenced on 1 June 2021 (the transitional prescribed period).
	(2)	The Minister may, by notice in the Gazette published before the date for payment of contributions for the transitional prescribed period, vary an amount fixed under regulation 5 of the revoked regulations in respect of that period.

Initialize CrewAI

In [15]:
from crewai import Agent, Task, Crew, Process
from graphlit_tools import PromptTool, CrewAIConverter
from textwrap import dedent

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# NOTE: providing LLM specification which specifies the model to use for LLM completion
prompt_tool = CrewAIConverter.from_tool(PromptTool(graphlit, specification_id=specification_id))

# Agents
legal_review_agent = Agent(
    role="Senior Legal Document Reviewer",
    goal="Review and summarize legal documents, flag important clauses, and retrieve relevant case law.",
    backstory=(
        "You are an experienced legal professional with expertise in Australian legislation "
        "and case law. You have been tasked with reviewing the legal obligations of a Clare Valley "
        "winemaker who has underpaid their contributions to the Fund."
    ),
    verbose=True,
    tools=[prompt_tool],
    max_execution_time=60,
    max_rpm=None,
)

compliance_agent = Agent(
    role="Legal Compliance Expert",
    goal="Identify and flag compliance-related issues, ensuring adherence to legal standards.",
    backstory=(
        "You specialize in legal compliance, with a focus on ensuring that organizations "
        "meet their regulatory obligations. Your task is to assess the compliance risks "
        "associated with the winemaker's underpayment and determine how to rectify the issue."
    ),
    verbose=True,
    tools=[prompt_tool],
    max_execution_time=60,
    max_rpm=None,
)

case_law_researcher = Agent(
    role="Case Law Researcher",
    goal="Retrieve relevant case law and provide context for legal decisions.",
    backstory=(
        "You are a legal researcher focused on finding and analyzing relevant legal precedents. "
        "In this case, you need to identify any case law that provides guidance on similar compliance "
        "issues within the wine industry or related sectors."
    ),
    verbose=True,
    tools=[prompt_tool],
    max_execution_time=60,
    max_rpm=None,
)

legal_drafter = Agent(
    role="Legal Drafting Specialist",
    goal="Draft legal documents that are compliant with legal standards and aligned with organizational goals.",
    backstory=(
        "You are a skilled legal drafter responsible for creating precise and legally sound documents. "
        "In this task, you will draft a formal response on behalf of the winemaker to the regulatory body, "
        "addressing the underpayment issue and proposing a compliant solution."
    ),
    verbose=True,
    max_execution_time=60,
    max_rpm=None,
)

# Tasks
review_task = Task(
    description=(
        "Review the Clare Valley Wine Industry Fund regulations and summarize the key points. "
        "Flag any important clauses or terms that require further attention, particularly those "
        "related to underpayment and compliance."
    ),
    expected_output="Summarized report with flagged clauses.",
    agent=legal_review_agent
)

compliance_task = Task(
    description=(
        "Analyze the summary and identify any compliance issues or potential risks associated with "
        "the winemaker's underpayment. Provide recommendations on how to rectify the situation."
    ),
    expected_output="Compliance report with risk assessment and recommendations.",
    agent=compliance_agent,
    context=[review_task]
)

case_law_task = Task(
    description=(
        "Retrieve relevant case law that may provide guidance on similar compliance issues in the wine industry. "
        "Analyze how these precedents might impact the current legal situation."
    ),
    expected_output="Case law report with analysis.",
    agent=case_law_researcher,
    context=[compliance_task]
)

drafting_task = Task(
    description=(
        "Draft a formal legal response on behalf of the winemaker to the regulatory body. "
        "The document should address the underpayment issue, propose a compliant solution, "
        "and reference relevant regulations and case law."
    ),
    expected_output="Drafted legal document in markdown format of max 2000 characters.",
    agent=legal_drafter,
    context=[case_law_task]
)

# Crew process
crew = Crew(
    agents=[legal_review_agent, compliance_agent, case_law_researcher, legal_drafter],
    tasks=[review_task, compliance_task, case_law_task, drafting_task],
    verbose=True,
    planning=True,
    process=Process.sequential,
    memory=True
)

# Kick off the process
result = await crew.kickoff_async()

print(result)

 
[2024-12-11 09:00:23][INFO]: Planning the crew execution
# Agent: Senior Legal Document Reviewer
## Task: Review the Clare Valley Wine Industry Fund regulations and summarize the key points. Flag any important clauses or terms that require further attention, particularly those related to underpayment and compliance.1. **Gather the Document**: Obtain the latest version of the Clare Valley Wine Industry Fund regulations to ensure accuracy. 
2. **Initial Reading**: Conduct a quick read-through of the document to familiarize with its structure and content. 
3. **Deep Analysis**: Utilize the CrewAIConverter (Graphlit RAG prompt tool) by inputting a prompt that asks for a detailed summarization of the key points within the regulations. 
4. **Flag Relevant Clauses**: Specifically ask the tool to highlight any clauses or terms that pertain to underpayment and compliance issues, marking them for further attention. 
5. **Compile the Summarized Report**: Organize the summarized content and flag

ERROR:asyncio:Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x79ab1daaeec0> is already entered




# Agent: Legal Compliance Expert
## Thought: I need to analyze the summarized report to identify compliance issues and risks associated with the winemaker's underpayment. I will utilize the Graphlit RAG prompt tool to gather the necessary information for the analysis.
## Using tool: Graphlit RAG prompt tool
## Tool Input: 
"{\"prompt\": \"Identify compliance issues related to underpayment based on the summary of the Clare Valley Wine Industry Fund Regulations. Assess potential risks associated with these issues and provide actionable recommendations for winemakers to rectify underpayment situations.\"}"
## Tool Output: 
Compliance issues related to underpayment in the Clare Valley Wine Industry Fund Regulations include failing to remit the required contributions by the stipulated deadlines, inaccurately calculating contribution amounts based on the volume of grapes delivered, and not maintaining proper records of grape deliveries and corresponding contributions. Additionally, winemak

ERROR:asyncio:Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x79ab1d9ec480> is already entered
ERROR:asyncio:Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x79ab1dab3240> is already entered
ERROR:asyncio:Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run



# Agent: Case Law Researcher
## Using tool: Graphlit RAG prompt tool
## Tool Input: 
"{\"prompt\": \"Retrieve relevant case law examples concerning compliance issues related to underpayment in the wine industry and similar agricultural sectors. Focus on cases that highlight failures in remitting contributions, inaccuracies in contribution calculations, and record-keeping deficiencies. Analyze how these cases may impact current compliance issues faced by winemakers in the Clare Valley Wine Industry Fund.\"}"
## Tool Output: 
Several cases highlight compliance issues similar to those faced by winemakers in the Clare Valley Wine Industry Fund. For instance, in Kamha v Australian Prudential Regulation Authority [2005] FCA 480, the court addressed failures in remitting required contributions, emphasizing the importance of accurate financial reporting and adherence to regulatory mandates. Additionally, Corbett v Toll Stevedoring Pty Ltd & Ors [2007] NSWSC 749 demonstrated the repercussions